In [1]:
import pandas as pd
import datetime
import numpy as np
import os
import glob
import gc
import logging
logging.basicConfig(filename='weekly_audience_tracker.log', level=logging.INFO)

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
os.getcwd()
print("Worker Start: "+str(datetime.datetime.now()))

Worker Start: 2019-11-07 11:16:29.200756


In [2]:
last_sturday = (datetime.datetime.now()-datetime.timedelta(days=(datetime.datetime.now().weekday()+2))).date()
print("last_sturday: "+str(last_sturday))

last_day_of_2018Q4=datetime.date(2019,2,2)

year_of_quarter=(last_sturday-last_day_of_2018Q4).days/(52*7)
year_of_quarter=str(int(2019+np.floor(year_of_quarter)))
print("Year",year_of_quarter)


quarter_of_quarter=(last_sturday-last_day_of_2018Q4).days/7
quarter_of_quarter=np.floor(quarter_of_quarter/13)%4
quarter_of_quarter=str(int(1+quarter_of_quarter))
print("Quarter",quarter_of_quarter)

str_current_quarter=year_of_quarter+"_Q"+quarter_of_quarter

print(str_current_quarter)

current_week=int((last_sturday-last_day_of_2018Q4).days/7%13)
print("current_week",current_week)

if current_week==0:
    quarter_of_quarter=int(quarter_of_quarter)-1
    str_current_quarter=year_of_quarter+"_Q"+str(quarter_of_quarter)
    current_week=13
    print("Quarter",quarter_of_quarter)
    print(str_current_quarter)
    print("current_week",current_week)
    
    
quarter_of_quarter


# In[3]:

current_quarter_beginning=last_day_of_2018Q4+datetime.timedelta(days=(int(year_of_quarter)-2019)*52*7+                                                                (int(quarter_of_quarter)-1)*13*7+1)
print("current_quarter_beginning: "+str(current_quarter_beginning))

current_quarter_beginning_last_year=current_quarter_beginning-datetime.timedelta(days=52*7)
print("current_quarter_beginning_last_year: "+str(current_quarter_beginning_last_year))


# In[4]:

all_weekly_new_sign_ups=list(recursive_file_gen("/home/jian/BigLots/"))
all_weekly_new_sign_ups=[x for x in all_weekly_new_sign_ups if "aster" in x.lower()]
all_weekly_new_sign_ups=[x for x in all_weekly_new_sign_ups if x[-4:]==".txt"]
all_weekly_new_sign_ups=[x for x in all_weekly_new_sign_ups if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning_last_year)]
all_weekly_new_sign_ups.sort()
print(len(all_weekly_new_sign_ups))

new_sign_ups_piece_1="/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip"
new_sign_ups_piece_2="/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip"
all_weekly_new_sign_ups.append(new_sign_ups_piece_1)
all_weekly_new_sign_ups.append(new_sign_ups_piece_2)
print(len(all_weekly_new_sign_ups))


# In[5]:

list_new_ids=[]
for file in all_weekly_new_sign_ups:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    df=df[df['sign_up_date']>=str(current_quarter_beginning_last_year)]
    list_new_ids=list_new_ids+df['customer_id_hashed'].unique().tolist()
list_new_ids=list(set(list_new_ids))


# In[6]:

audience_files_folder="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_"+str_current_quarter.replace("_","")+"/upload_files/"


audience_files_this_quarter=glob.glob(audience_files_folder+"*.csv")
len(audience_files_this_quarter)


audience_files_this_quarter_C=[x for x in audience_files_this_quarter if os.path.basename(x)[0]=="C"]
audience_files_this_quarter_T=[x for x in audience_files_this_quarter if os.path.basename(x)[0]=="T"]
print(len(audience_files_this_quarter_C),len(audience_files_this_quarter_T))

set_Control=set()
for file in audience_files_this_quarter_C:
    df=pd.read_csv(file,dtype=str,usecols=["customer_id_hashed"])
    set_Control=set_Control.union(set(df['customer_id_hashed'].tolist()))
print(len(set_Control))


df_audience=pd.DataFrame()

for file in audience_files_this_quarter_T:
    df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','segment'])
    df=df[~df['customer_id_hashed'].isin(set_Control)]
    df_audience=df_audience.append(df)
df_audience=df_audience.drop_duplicates()

print(df_audience.shape," | ",df_audience['customer_id_hashed'].nunique())


# In[7]:

# Rolling up removing the quadrant
# Depends on the segment naming
df_audience_total_count_original=df_audience.groupby("segment")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})
df_audience['segment_2']=df_audience['segment'].str.replace("Quadrant I_","")
df_audience['segment_2']=df_audience['segment_2'].str.replace("Quadrant II_","")
df_audience['segment_2']=df_audience['segment_2'].str.replace("Quadrant III_","")
df_audience['segment_2']=df_audience['segment_2'].str.replace("Quadrant IV_","")
print(df_audience.shape)

df_audience_total_count=df_audience.groupby("segment_2")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})

del df_audience['segment']
df_audience=df_audience.rename(columns={"segment_2":"segment"})

print(df_audience_total_count.shape)
df_audience_total_count


# In[8]:

list_POS_this_year_in_quarter=list(recursive_file_gen("/home/jian/BigLots/"))
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "Daily" in x and x[-4:]==".txt"]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2016" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2017" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2018" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "histor" not in x.lower()]
list_POS_this_year_in_quarter.sort()
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning)]
# date only >=
print("len(list_POS_this_year_in_quarter): "+str(len(list_POS_this_year_in_quarter)))


last_sturday: 2019-11-02
Year 2019
Quarter 4
2019_Q4
current_week 0
Quarter 3
2019_Q3
current_week 13
current_quarter_beginning: 2019-08-04
current_quarter_beginning_last_year: 2018-08-05
53
55
25 25
542702
(21810554, 2)  |  21810554
(21810554, 3)
(7, 2)
len(list_POS_this_year_in_quarter): 13


In [15]:
last_year_end_week_in_quarter=last_sturday-datetime.timedelta(days=52*7)

if year_of_quarter=="2019" and (quarter_of_quarter==3 or quarter_of_quarter==4): 
    list_POS_last_year_in_quarter=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStormDailySalesHistory")[1][:8]>=str(current_quarter_beginning_last_year).replace("-","")]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStormDailySalesHistory")[1][:8]<=str(last_year_end_week_in_quarter).replace("-","")]

elif year_of_quarter=="2020" and quarter_of_quarter==1: 
    list_POS_last_year_in_quarter=list(recursive_file_gen("/home/jian/BigLots/"))
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "Daily" in x and x[-4:]==".txt"]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "history" not in x.lower()]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2019" in x]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2019-02-09" not in x]

    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning_last_year)]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStorm_")[1][:10]<=str(last_year_end_week_in_quarter)]
    list_POS_last_year_in_quarter.append("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt,1156035811,1551304443000")
else:
    list_POS_last_year_in_quarter=list(recursive_file_gen("/home/jian/BigLots/"))
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "Daily" in x and x[-4:]==".txt"]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2018" not in x]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2017" not in x]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2016" not in x]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "history" not in x.lower()]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning_last_year)]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStorm_")[1][:10]<=str(last_year_end_week_in_quarter)]


# In[10]:

list_POS_this_year_in_quarter=list(recursive_file_gen("/home/jian/BigLots/"))
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "Daily" in x and x[-4:]==".txt"]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2016" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2017" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2018" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "histor" not in x.lower()]
list_POS_this_year_in_quarter.sort()
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning)]
# date only >=
print("len(list_POS_this_year_in_quarter): "+str(len(list_POS_this_year_in_quarter)))


# In[11]:

if len(list_POS_this_year_in_quarter)!=current_week or len(list_POS_last_year_in_quarter)!=current_week:
    print(str(datetime.datetime.now())+" | Error: POS file lenthg this year and last year different")
    print(str(datetime.datetime.now())+" | current_week "+str(current_week))
    
    print(str(datetime.datetime.now())+" | list_POS_this_year_in_quarter "+str(len(list_POS_this_year_in_quarter)))
    print(str(datetime.datetime.now())+" | list_POS_last_year_in_quarter "+str(len(list_POS_last_year_in_quarter)))
    # break


# In[12]:

print(datetime.datetime.now())


# In[13]:

list_sales_this_year=[]
list_sales_last_year=[]

len(list_POS_this_year_in_quarter): 13
2019-11-07 12:04:38.359101


In [14]:
year_of_quarter=="2019" and (quarter_of_quarter==3 or quarter_of_quarter==4)

True

In [11]:
list_POS_last_year_in_quarter=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStormDailySalesHistory")[1][:8]>=str(current_quarter_beginning_last_year).replace("-","")]
list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStormDailySalesHistory")[1][:8]<=str(last_year_end_week_in_quarter).replace("-","")]

In [17]:
len(list_POS_last_year_in_quarter)

13